In [9]:
# Required libraries
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
from pandas_datareader import data as pdr

# ---------------------------
# 1. Download stock data
# ---------------------------
stock_data = yf.download("AAPL", start="2020-01-01", end="2025-01-01")

# MultiIndex: pick Close prices
stock = stock_data['Close']['AAPL']

# Compute daily returns
stock_ret = stock.pct_change().dropna()

# ---------------------------
# 2. Download Fama-French 3 factors (daily)
# ---------------------------
ff3 = pdr.DataReader("F-F_Research_Data_Factors_daily", "famafrench")[0]
ff3.index = pd.to_datetime(ff3.index)  # ensure datetime index

# ---------------------------
# 3. Align stock returns with FF factors
# ---------------------------
data = pd.merge(stock_ret, ff3, left_index=True, right_index=True)
data.rename(columns={stock_ret.name: "Stock"}, inplace=True)

# Compute excess stock return
# FF factors are in %, so convert stock returns to %
data["Excess_Stock"] = data["Stock"]*100 - data["RF"]

# ---------------------------
# 4. Run Fama-French 3-Factor regression
# ---------------------------
X = data[["Mkt-RF", "SMB", "HML"]]
X = sm.add_constant(X)  # adds alpha
y = data["Excess_Stock"]

model = sm.OLS(y, X).fit()
print(model.summary())

# ---------------------------
# 5. Extract factor loadings
# ---------------------------
alpha = model.params['const']
beta_mkt = model.params['Mkt-RF']
beta_smb = model.params['SMB']
beta_hml = model.params['HML']

print(f"\nAlpha: {alpha:.4f}, Beta Market: {beta_mkt:.4f}, Beta SMB: {beta_smb:.4f}, Beta HML: {beta_hml:.4f}")


/var/folders/c0/_b6c00gj3x56bbmcgtgs73gr0000gn/T/ipykernel_73666/3017185548.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download("AAPL", start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed


/var/folders/c0/_b6c00gj3x56bbmcgtgs73gr0000gn/T/ipykernel_73666/3017185548.py:21: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff3 = pdr.DataReader("F-F_Research_Data_Factors_daily", "famafrench")[0]


                            OLS Regression Results                            
Dep. Variable:           Excess_Stock   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.607
Method:                 Least Squares   F-statistic:                     558.1
Date:                Sat, 13 Sep 2025   Prob (F-statistic):          7.95e-219
Time:                        17:42:24   Log-Likelihood:                -1615.3
No. Observations:                1082   AIC:                             3239.
Df Residuals:                    1078   BIC:                             3258.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0312      0.033      0.949      0.3